In [1]:
from carbon.helpers.stdimports import *
from carbon.helpers.soltest import SolTestBase
from carbon.helpers.floatint import *
#print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SolTestBase))
#print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonFloatInt32))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonOrderUI))
#print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(P))
from math import log2, floor, ceil, sqrt
print_version(require="2.3.3")

[stdimports] imported np, pd, plt, os, sqrt, exp, log
CarbonOrderUI v1.7.1 (12/Feb/2023)
Carbon v2.3.3-BETA1 (12/Feb/2022)


# Reconciliations (NBTest905)

## Reconcile CarbonOrderUI vs Core Sim

In [85]:
Sim = CarbonSimulatorUI(pair=P("ETH/USDC"), raiseonerror=True)
r = Sim.add_strategy("ETH", 10, 1500, 2000, 1000, 750, 500)
r["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,False,1500.0,2000.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,USDC,1000.0,1000.0,USDC,False,750.0,500.0,750.0,USDC per ETH,0


In [86]:
ouis=r["orderuis"]
ouis

{0: CarbonOrderUI(pair=P('ETH/USDC'), tkn=ETH, B=0.022360679774997897, S=0.0034592091997182155, yint=10.0, y=10.0, id=0, linked=<1>),
 1: CarbonOrderUI(pair=P('ETH/USDC'), tkn=USDC, B=22.360679774997898, S=5.025448100260409, yint=1000.0, y=1000.0, id=1, linked=<0>)}

### amm_sells ETH (dx from dy)

In [95]:
size = 0.1
r = Sim.amm_sells("ETH", size, execute=False)
tr = r["trades"].query("aggr==True").iloc[0]
r = Sim.amm_sells("ETH", size, execute=False)
tr2 = r["trades"].query("aggr==True").iloc[0]
assert tr["price"] == tr2["price"] # check trades are NOT being executed
dx = ouis[0].dxfromdy_f(size)
assert abs(dx/tr["amt2"] - 1) < 1e-10
tr2["amt2"], dx

(150.201231493404, 150.20123149340415)

In [97]:
for size in np.linspace(0.1,1,10):
    r = Sim.amm_sells("ETH", size, execute=False)
    tr = r["trades"].query("aggr==True").iloc[0]
    amts = tr["amt2"]
    dx = ouis[0].dxfromdy_f(size)
    print(size, amts, dx, amts/dx-1)
    assert abs(amts/dx - 1) < 1e-10

0.1 150.201231493404 150.20123149340415 -9.992007221626409e-16
0.2 300.806007267257 300.80600726725703 -2.220446049250313e-16
0.30000000000000004 451.815955807291 451.815955807291 0.0
0.4 603.232714373287 603.2327143732872 -3.3306690738754696e-16
0.5 755.057929058247 755.0579290582469 2.220446049250313e-16
0.6 907.293254848042 907.2932548480424 -3.3306690738754696e-16
0.7000000000000001 1059.940355681551 1059.9403556815514 -4.440892098500626e-16
0.8 1213.000904511281 1213.0009045112809 2.220446049250313e-16
0.9 1366.476583364485 1366.4765833644851 -1.1102230246251565e-16
1.0 1520.369083404778 1520.3690834047782 -1.1102230246251565e-16


### amm_sells USDC (dx from dy)

In [108]:
for size in np.linspace(0.1,1000,10):
    r = Sim.amm_sells("USDC", size, execute=False)
    tr = r["trades"].query("aggr==True").iloc[0]
    amts = tr["amt2"]
    dx = ouis[1].dxfromdy_f(size)
    print(size, amts, dx, amts/dx-1)
    assert abs(amts/dx - 1) < 1e-8

0.1 0.00013333578 0.00013333578009048648 -6.786361472777003e-10
111.19999999999999 0.15135515645 0.15135515645042602 -2.8147484343321594e-12
222.29999999999998 0.309005190024 0.3090051900241366 -4.4209080840573733e-13
333.4 0.473502217928 0.47350221792774383 5.411227022023013e-13
444.5 0.645302205773 0.6453022057730544 -8.426592756904938e-14
555.6 0.824902524093 0.8249025240928146 2.2470914018413168e-13
666.6999999999999 1.01284675734 1.012846757340231 -2.282618538629322e-13
777.8 1.209730199081 1.2097301990811244 -1.028066520802895e-13
888.9 1.416206150393 1.4162061503925296 3.3217872896784684e-13
1000.0 1.632993161855 1.6329931618554518 -2.76667577736589e-13


### amm_buys USDC (dy from dx)

In [122]:
for size in np.linspace(0.1,1000,10):
    r = Sim.amm_buys("USDC", size, execute=False)
    tr = r["trades"].query("aggr==True").iloc[0]
    amts = tr["amt1"]
    dx = ouis[0].dyfromdx_f(size)
    print(size, amts, dx, amts/dx-1)
    assert abs(amts/dx - 1) < 1e-8

0.1 6.6666607e-05 6.666660712245485e-05 -1.8368243592803424e-09
111.19999999999999 0.074059777285 0.0740597772854767 -6.43674002986927e-12
222.29999999999998 0.147906331459 0.14790633145886237 9.305889392408062e-13
333.4 0.221606764118 0.22160676411829475 -1.3300471835009375e-12
444.5 0.295161508535 0.2951615085350415 -1.404432126150823e-13
555.6 0.368570996269 0.3685709962691127 -3.0586644328423063e-13
666.6999999999999 0.441835657178 0.44183565717770173 6.750155989720952e-13
777.8 0.514955919424 0.5149559194235748 8.257838857161914e-13
888.9 0.587932209483 0.5879322094834122 -7.011058400507864e-13
1000.0 0.660764952156 0.6607649521560994 -1.5054624213917123e-13


### amm_buys ETH (dy from dx)

In [125]:
for size in np.linspace(0.1,1,10):
    r = Sim.amm_buys("ETH", size, execute=False)
    tr = r["trades"].query("aggr==True").iloc[0]
    amts = tr["amt1"]
    dx = ouis[1].dyfromdx_f(size)
    print(size, amts, dx, amts/dx-1)
    assert abs(amts/dx - 1) < 1e-8

0.1 73.981806417987 73.98180641798673 3.552713678800501e-15
0.2 145.981776727246 145.98177672724577 1.5543122344752192e-15
0.30000000000000004 216.078492993245 216.07849299324494 2.220446049250313e-16
0.4 284.346436983739 284.34643698373947 -1.5543122344752192e-15
0.5 350.856254159561 350.85625415956116 -4.440892098500626e-16
0.6 415.674997528775 415.6749975287748 4.440892098500626e-16
0.7000000000000001 478.866353133682 478.8663531336817 6.661338147750939e-16
0.8 540.490848764647 540.4908487646471 -2.220446049250313e-16
0.9 600.606047338574 600.6060473385742 -3.3306690738754696e-16
1.0 659.266726240687 659.2667262406868 4.440892098500626e-16


## Verifying equivalence of both order sets

In [13]:
Sim = CarbonSimulatorUI(pair=P("ETH/USDC"), raiseonerror=True)
Sim.add_strategy("ETH", 1, 1500, 2000, 1000, 750, 500)
Sim.add_strategy("ETH", 1.1, 1501, 2001, 1001, 751, 501)
r = Sim.state()
os = r["orders"]
os

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,1.0,1.0,ETH,False,1500.0,2000.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,USDC,1000.0,1000.0,USDC,False,750.0,500.0,750.0,USDC per ETH,0
2,2,ETHUSDC,ETH,1.1,1.1,ETH,False,1501.0,2001.0,1501.0,USDC per ETH,3
3,3,ETHUSDC,USDC,1001.0,1001.0,USDC,False,751.0,501.0,751.0,USDC per ETH,2


In [11]:
ouis = r["orderuis"]
ouis

{0: CarbonOrderUI(pair=P('ETH/USDC'), tkn=ETH, B=0.022360679774997897, S=0.0034592091997182155, yint=1.0, y=1.0, id=0, linked=<1>),
 1: CarbonOrderUI(pair=P('ETH/USDC'), tkn=USDC, B=22.360679774997898, S=5.025448100260409, yint=1000.0, y=1000.0, id=1, linked=<0>),
 2: CarbonOrderUI(pair=P('ETH/USDC'), tkn=ETH, B=0.022355091700494795, S=0.003456194945488572, yint=1.1, y=1.1, id=2, linked=<3>),
 3: CarbonOrderUI(pair=P('ETH/USDC'), tkn=USDC, B=22.38302928559939, S=5.02134992649005, yint=1001.0, y=1001.0, id=3, linked=<2>)}

In [47]:
assert len(ouis)==len(os)
for oui, oo in zip(ouis.values(),os.iterrows()):
    o = oo[1]
    #print (oui, o)
    assert o["id"] == oui.id
    assert o["lid"] == oui.lid
    assert o["pair"] == oui.pair.pair_iso
    assert o["tkn"] == oui.tkn
    assert o["tkn"] == oui.tkn
    assert o["y_int"] == oui.yint
    assert abs(o["p_start"]/oui.pa-1) < 1e-10
    assert abs(o["p_end"]/oui.pb-1) < 1e-10
    assert abs(o["y_int"]/oui.yint-1) < 1e-10
    assert abs(o["y"]/oui.y-1) < 1e-10
    assert o["p_unit"] == oui.price_convention
    assert oui.linked.linked is oui